In [89]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [90]:
import warnings
warnings.filterwarnings("ignore")

# Feature Selection with Machine Learning Alrogithms

## Remaining Tasks

- Interpretation of results of Neural Network
- Super Learner Ensemble: research, figure out specifics of how used in corruption paper, implement
- Run each GridSearch 10x and average the final predictors
- Finish data cleaning (ironore/silver variables issue, remaining NAs from year_deforest, repeat in jobs from analysis.Rdata and more specific jobs ones, are there too many job categories?)
- what are n_jobs_ and n_companies_
- once i have the final variable list sorted out, consider model performance score MSE and also covariate groups of variables, see how the paper found the groups and try to replicate

- get rid of jobs overlap variables
- step ahead forecasting

- feature importance while holding certain variables constant (e.g. lat/long)

Methods Used:
- Univariate Test
- RFE
- PCA
- Random Forests * use grid search
- Extra Trees * use grid search
- Lasso Regression ** use grid search
- Gradient Boosting Machine ** use grid search
- Neural Network
- Super Learner Ensemble

Paper Methods
- LASSO (Done)
- Random Forests (Done)
- Gradient Boosting Machine
- Neural Network
- Super Learner Ensemble

## Cross Validation

CV is applied to relevant methods with sklearn's GridSearchCV package. GridSearch tests model performance on all possible values of hyperparameters. 5-fold CV is used here as in the paper. 
- Criterion selected for CV is neg mse instead of their classification-based metric

- What I don't have yet is that they repeat the CV 10 times and use the average of those parameters. Need to write a function and run this. Will probably be better to do this on yale compute

## SpatioTemporal CV

In [257]:
import spacv
from spacv.grid_builder import *
from spacv.utils import *

In [258]:
from sklearn.svm import SVC

## Import Data and PreProcess

In [91]:
#path = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/full_data_2016.csv'
#path = 'TempFullData.csv'
path = 'Dec22FullData.csv'
df1 = pd.read_csv(path, index_col = [0])

In [92]:
df1.shape[0]

2455930

In [93]:
#drop nas for now

df1 = df1[~df1.expen_other_agr.isna()]

In [94]:
dict(sorted((df1.isna().sum()).to_dict().items(), key=lambda item: item[1]))

{'ID': 0,
 'forest.diff': 0,
 'FID': 0,
 'x': 0,
 'y': 0,
 'year': 0,
 'forest.l': 0,
 'nn_forest.l': 0,
 'rain1': 0,
 'elevation': 0,
 'slope': 0,
 'aspect': 0,
 'near_mines': 0,
 'near_roads': 0,
 'near_hidrovia': 0,
 'indigenous_homol': 0,
 'mun_election_year': 0,
 'new_forest_code': 0,
 'lula': 0,
 'dilma': 0,
 'temer': 0,
 'bolsonaro': 0,
 'fed_election_year': 0,
 'populacao': 0,
 'pib_pc': 0,
 'ironore': 0,
 'silver': 0,
 'copper': 0,
 'gold': 0,
 'soy_price': 0,
 'beef_price': 0,
 'ag_jobs': 0,
 'mining_jobs': 0,
 'public_jobs': 0,
 'construction_jobs': 0,
 'PIB': 0,
 'n_companies_PUBLIC ADMIN': 0,
 'n_companies_AGRICULTURE': 0,
 'n_companies_FOOD AND DRINKS': 0,
 'n_companies_ACCOMODATION AND FOOD': 0,
 'n_companies_EQUIPMENT RENTAL': 0,
 'n_companies_WHOLESALE': 0,
 'n_companies_ASSOCIATIVE ACTIVITIES': 0,
 'n_companies_AUTOMOBILES AND TRANSPORT': 0,
 'n_companies_FINANCIAL ASSISTANCE': 0,
 'n_companies_TRADE REP VEHICLES': 0,
 'n_companies_CONSTRUCTION': 0,
 'n_companies_MAIL

In [95]:
#df = df1[df1.year == 2016]
df = df1

In [96]:
df.shape

(2455780, 167)

In [97]:
df.head()

,ID,forest.diff,FID,x,y,year,forest.l,nn_forest.l,rain1,elevation,...,expen_other_agr,expen_agr_defense,expen_min_fuel,illegal_mining,illegal_other,illegal_industry,audits,emiss_pec_full,emiss_agr_full,emiss_agropec_full
0,1100015.0,0.0,8436645,-62.825,-12.925,2002,97.0,85.50,21.81664,154.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,488528.553429,5304.103889,4902816.0
1,1100015.0,16.0,8436646,-62.825,-12.875,2002,65.0,68.75,22.56391,146.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,488528.553429,5304.103889,4902816.0
2,1100015.0,5.0,8440244,-62.775,-12.975,2002,92.0,87.75,21.30135,156.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,488528.553429,5304.103889,4902816.0
3,1100015.0,9.0,8440245,-62.775,-12.925,2002,83.0,73.75,22.10585,156.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,488528.553429,5304.103889,4902816.0
4,1100015.0,13.0,8443844,-62.725,-12.975,2002,79.0,80.25,20.90614,155.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,488528.553429,5304.103889,4902816.0


In [98]:
df = df[['ID', 'forest.diff', 'FID', 'x', 'y', 'year', 'forest.l', 'rain1', 'elevation', 'slope', 'aspect',
       'near_mines', 'near_roads', 'near_hidrovia', 'indigenous_homol',
       'mun_election_year', 'new_forest_code', 'lula', 'dilma', 'temer',
       'bolsonaro', 'fed_election_year', 'populacao', 'pib_pc', 'ironore',
       'silver', 'copper', 'gold', 'soy_price', 'beef_price', 'ag_jobs',
       'mining_jobs', 'public_jobs', 'construction_jobs', 'PIB', 'n_companies_AGRICULTURE',
       'n_companies_CONSTRUCTION','n_companies_CLEANING AND SEWAGE', 'n_companies_MACHINERY', 'n_companies_MINING',
       'n_companies_WOOD PROD',
       'n_companies_NON-METALLIC MINERAL PRODUCTS', 'n_companies_HEALTH', 'n_companies_TRANSPORTATION',
       'n_companies_GROUND TRANSPORT',
       'n_companies_WATER TREATMENT AND DISTRIBUTION',
       'n_companies_METALLIC PRODUCTS',
       'n_companies_FORESTRY',
       'n_companies_PAPER', 
       'n_companies_OIL AND GAS',
       'n_companies_CHEMICALS', 
       'n_companies_REAL ESTATE', 'n_companies_RECYCLING',
       'n_companies_LEATHERS AND FOOTWEAR',
       'n_companies_RUBBER AND PLASTIC', 'n_companies_TEXTILES',
       'n_companies_AERO TRANSPORT', 'n_companies_SMOKE',
       'n_companies_PETROLEUM REFINING', 'n_companies_','n_jobs_AGRICULTURE','n_jobs_CONSTRUCTION','n_jobs_EDUCATION',
       'n_jobs_ELECTRICITY AND GAS', 
       'n_jobs_CLEANING AND SEWAGE', 'n_jobs_MACHINERY',
       'n_jobs_BASIC METALLURGY', 'n_jobs_MINING', 'n_jobs_WOOD PROD',
       'n_jobs_NON-METALLIC MINERAL PRODUCTS', 'n_jobs_HEALTH',
       'n_jobs_TRANSPORTATION', 'n_jobs_GROUND TRANSPORT',
       'n_jobs_WATER TREATMENT AND DISTRIBUTION', 
       'n_jobs_METALLIC PRODUCTS',
       'n_jobs_FORESTRY', 'n_jobs_PAPER', 'n_jobs_OIL AND GAS',
       'n_jobs_FISHING AND AQUACULTURE', 'n_jobs_CHEMICALS',
       'n_jobs_WATER-BASED TRANSPORTATION', 'n_jobs_REAL ESTATE',
       'n_jobs_RECYCLING', 'n_jobs_LEATHERS AND FOOTWEAR',
       'n_jobs_RUBBER AND PLASTIC', 'n_jobs_TEXTILES','n_jobs_AERO TRANSPORT',
       'n_jobs_SMOKE', 'n_jobs_PETROLEUM REFINING', 
       'n_jobs_TOTAL INDUSTRIAL', 'n_jobs_TOTAL SERVICE',
       'n_companies_TOTAL INDUSTRIAL', 'n_companies_TOTAL SERVICE',
       'n_companies_TOTAL', 'n_jobs_TOTAL', 'murder_threats',
       'assassination', 'assassination_attempt', 'f_emitted_count',
       'expen_agri', 'expen_env_man', 'expen_agr_org', 'expen_mining',
       'expen_petrol', 'expen_prom_ani_pro', 'expen_prom_veg_pro',
       'expen_other_agr', 'expen_agr_defense', 'expen_min_fuel',
       'illegal_mining', 'illegal_other', 'illegal_industry', 'audits',
       'emiss_pec_full', 'emiss_agr_full', 'emiss_agropec_full']]

In [99]:
#df = df.join(pd.get_dummies(df.genero, prefix = 'gender'))
#df = df.join(pd.get_dummies(df.ocupacao, prefix = 'occ'))
#df = df.join(pd.get_dummies(df.instrucao, prefix = 'edu'))
#df = df.join(pd.get_dummies(df.sigla_partido, prefix = 'party'))

In [100]:
#options:
#one year to predict the next
#one year for its own

In [720]:
explan_year = 2016
response_year = 2003

response_temp = df[df.year == response_year][['FID', 'forest.diff']]
response_temp.columns = ['FID', 'response_variable']

explan_df = pd.merge(df[df.year == explan_year], response_temp, how = 'inner', on = 'FID')

In [721]:
#response_df = df[df.year == 2016]

#explan_df = df[df.year == 2016]

#explan_df = explan_df[~explan_df.duplicated(subset=['FID'], keep = 'last')]
#response_df = response_df[~response_df.duplicated(subset=['FID'], keep = 'last')]

In [722]:
print(explan_df.shape[0])

160798


In [723]:
#explan_df = pd.merge(explan_df, response_df[['FID', 'forest.l']], how = 'left', on = 'FID')

In [724]:
explan_df.shape[0]

160798

In [725]:
np.unique(df.year)

array([2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016])

In [726]:
explan_df.head()

,ID,forest.diff,FID,x,y,year,forest.l,rain1,elevation,slope,...,expen_agr_defense,expen_min_fuel,illegal_mining,illegal_other,illegal_industry,audits,emiss_pec_full,emiss_agr_full,emiss_agropec_full,response_variable
0,1100015.0,0.0,8436645,-62.825,-12.925,2016,87.0,22.84262,154.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,753215.23692,8479.297773,761694.534692,-13.0
1,1100015.0,16.0,8436646,-62.825,-12.875,2016,75.0,23.28390,146.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,753215.23692,8479.297773,761694.534692,-33.0
2,1100015.0,0.0,8440244,-62.775,-12.975,2016,84.0,22.99245,156.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,753215.23692,8479.297773,761694.534692,-5.0
3,1100015.0,0.0,8440245,-62.775,-12.925,2016,97.0,23.08443,156.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,753215.23692,8479.297773,761694.534692,-14.0
4,1100015.0,-1.0,8443844,-62.725,-12.975,2016,92.0,22.24266,155.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,753215.23692,8479.297773,761694.534692,-15.0


In [727]:
Y = explan_df['response_variable']

X = explan_df[['rain1', 'elevation', 'slope', 'aspect',
       'near_mines', 'near_roads', 'near_hidrovia', 'indigenous_homol',
       'mun_election_year', 'new_forest_code', 'lula', 'dilma', 'temer',
       'bolsonaro', 'fed_election_year', 'populacao', 'pib_pc', 'ironore',
       'silver', 'copper', 'gold', 'soy_price', 'beef_price', 'ag_jobs',
       'mining_jobs', 'public_jobs', 'construction_jobs', 'PIB', 'n_companies_AGRICULTURE',
       'n_companies_CONSTRUCTION','n_companies_CLEANING AND SEWAGE', 'n_companies_MACHINERY', 'n_companies_MINING',
       'n_companies_WOOD PROD',
       'n_companies_NON-METALLIC MINERAL PRODUCTS', 'n_companies_HEALTH', 'n_companies_TRANSPORTATION',
       'n_companies_GROUND TRANSPORT',
       'n_companies_WATER TREATMENT AND DISTRIBUTION',
       'n_companies_METALLIC PRODUCTS',
       'n_companies_FORESTRY',
       'n_companies_PAPER', 
       'n_companies_OIL AND GAS',
       'n_companies_CHEMICALS', 
       'n_companies_REAL ESTATE', 'n_companies_RECYCLING',
       'n_companies_LEATHERS AND FOOTWEAR',
       'n_companies_RUBBER AND PLASTIC', 'n_companies_TEXTILES',
       'n_companies_AERO TRANSPORT', 'n_companies_SMOKE',
       'n_companies_PETROLEUM REFINING', 'n_companies_','n_jobs_AGRICULTURE','n_jobs_CONSTRUCTION','n_jobs_EDUCATION',
       'n_jobs_ELECTRICITY AND GAS', 
       'n_jobs_CLEANING AND SEWAGE', 'n_jobs_MACHINERY',
       'n_jobs_BASIC METALLURGY', 'n_jobs_MINING', 'n_jobs_WOOD PROD',
       'n_jobs_NON-METALLIC MINERAL PRODUCTS', 'n_jobs_HEALTH',
       'n_jobs_TRANSPORTATION', 'n_jobs_GROUND TRANSPORT',
       'n_jobs_WATER TREATMENT AND DISTRIBUTION', 
       'n_jobs_METALLIC PRODUCTS',
       'n_jobs_FORESTRY', 'n_jobs_PAPER', 'n_jobs_OIL AND GAS',
       'n_jobs_FISHING AND AQUACULTURE', 'n_jobs_CHEMICALS',
       'n_jobs_WATER-BASED TRANSPORTATION', 'n_jobs_REAL ESTATE',
       'n_jobs_RECYCLING', 'n_jobs_LEATHERS AND FOOTWEAR',
       'n_jobs_RUBBER AND PLASTIC', 'n_jobs_TEXTILES','n_jobs_AERO TRANSPORT',
       'n_jobs_SMOKE', 'n_jobs_PETROLEUM REFINING', 
       'n_jobs_TOTAL INDUSTRIAL', 'n_jobs_TOTAL SERVICE',
       'n_companies_TOTAL INDUSTRIAL', 'n_companies_TOTAL SERVICE',
       'n_companies_TOTAL', 'n_jobs_TOTAL', 'murder_threats',
       'assassination', 'assassination_attempt', 'f_emitted_count',
       'expen_agri', 'expen_env_man', 'expen_agr_org', 'expen_mining',
       'expen_petrol', 'expen_prom_ani_pro', 'expen_prom_veg_pro',
       'expen_other_agr', 'expen_agr_defense', 'expen_min_fuel',
       'illegal_mining', 'illegal_other', 'illegal_industry', 'audits',
       'emiss_pec_full', 'emiss_agr_full', 'emiss_agropec_full']]

In [728]:
#test train split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state=42)

In [729]:
X_corr = X

## Covariance

In [730]:
#corr = X.corr()
#corr.style.background_gradient(cmap = 'coolwarm')

## NMF

In [731]:
from sklearn.decomposition import NMF

In [732]:
X.min().min()

0.0

In [733]:
Y_shift = Y + X.min().min()

In [734]:
X_shift = X + abs(X.min().min())

In [735]:
X.min().min()

0.0

In [736]:
#instantiate NMF model and fit to our X matrix
model = NMF(n_components=4)
model.fit(X_shift)

#extract W and H from the model that we've just fit
W = model.transform(X_shift)
H = model.components_

In [737]:
# Create a DataFrame: components_df, this is H, topics are the rows
H_df = pd.DataFrame(H, columns = X_shift.columns)
H_df

,rain1,elevation,slope,aspect,near_mines,near_roads,near_hidrovia,indigenous_homol,mun_election_year,new_forest_code,...,expen_other_agr,expen_agr_defense,expen_min_fuel,illegal_mining,illegal_other,illegal_industry,audits,emiss_pec_full,emiss_agr_full,emiss_agropec_full
0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,20.912898,0.000000,0.000000e+00,0.000000,0.000000e+00
1,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.646616,0.000000,0.000000e+00,33.252932,0.000000e+00
2,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.026088,0.000000e+00,307.745374,0.000000e+00
3,414.814243,3274.155516,11.449276,2521.216184,1.969430e+06,958343.839359,1.001703e+06,3.032938,14.402132,14.402132,...,235754.356991,148441.700511,0.0,17066.745807,0.259736,6439.084911,4.881304,1.050407e+07,509267.537288,1.102014e+07


In [738]:
for topic in range(H_df.shape[0]):
    tmp = H_df.iloc[topic]
    print(f'For basis vector {topic+1} the variables with the highest importance are:')
    print(tmp.nlargest(5))
    print('\n')

For basis vector 1 the variables with the highest importance are:
expen_env_man       2.445691e+06
PIB                 1.616115e+06
expen_agri          3.966520e+05
illegal_industry    2.091290e+01
murder_threats      5.121973e-01
Name: 0, dtype: float64


For basis vector 2 the variables with the highest importance are:
PIB                        9.308823e+06
expen_agri                 9.291976e+03
expen_env_man              7.088111e+03
emiss_agr_full             3.325293e+01
n_jobs_TOTAL INDUSTRIAL    7.657468e+00
Name: 1, dtype: float64


For basis vector 3 the variables with the highest importance are:
PIB              1.962589e+09
expen_agri       2.599691e+05
populacao        6.074320e+04
expen_env_man    3.181376e+04
n_jobs_TOTAL     1.548229e+04
Name: 2, dtype: float64


For basis vector 4 the variables with the highest importance are:
PIB                   1.665115e+10
gold                  5.783360e+08
expen_env_man         1.548868e+08
expen_agri            1.126116e+08
emi

## Random Forests

5.1.2. Random Forests. Random Forests are ensembles of many decision trees, where each one of them is a sequence of rules that divides the sample into sub-groups (called leaves) based on certain variable cutoffs. The prediction for each leaf, in the case of a classification task, is the most common outcome for the trained observations on that leaf, and the trees are fit so as to maximize the information gain of the resulting partitions of the data. Each tree in a Random Forest is constructed by sampling a random subset of the training data and a random subset of the predictors. Each of these trees generates a prediction, and the overall prediction of the Random Forest is the average (or the majority) of the predictions among all trees. **In this application, we keep fixed the number of fitted trees (500) and use cross-validation to determine the optimal number of features available in every node.**

**Here I tuned max_depth. This isn't exactly what they did, but there's not as much flexiblity in sklearn as far as I can tell**

For tree-based models, importance is measured as the information gain, or the homogeneity in the resulting partitions of our set of municipalities, achieved when splitting on each variable.

In [548]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler


#from sklearn.inspection import permutation_importance

In [549]:
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',RandomForestRegressor(n_estimators = 500))
])

In [550]:
search = GridSearchCV(pipeline,
                      {'model__max_depth':np.arange(3,11,8) },
                      cv = 5, scoring = "neg_mean_squared_error",verbose = 3
                      )

In [551]:
search.fit(X_train,Y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ..............model__max_depth=3;, score=-89.628 total time= 2.3min
[CV 2/5] END ..............model__max_depth=3;, score=-88.639 total time= 2.0min
[CV 3/5] END ..............model__max_depth=3;, score=-90.383 total time= 2.1min
[CV 4/5] END ..............model__max_depth=3;, score=-89.378 total time= 2.2min
[CV 5/5] END ..............model__max_depth=3;, score=-88.008 total time= 2.2min


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model',
                                        RandomForestRegressor(n_estimators=500))]),
             param_grid={'model__max_depth': array([3])},
             scoring='neg_mean_squared_error', verbose=3)

In [552]:
search.best_params_

{'model__max_depth': 3}

In [553]:
model = RandomForestRegressor(n_estimators = 500, max_depth = search.best_params_['model__max_depth'])
model.fit(X_train, Y_train)

RandomForestRegressor(max_depth=3, n_estimators=500)

In [554]:
importances = search.best_estimator_._final_estimator.feature_importances_[np.argsort(np.array(abs(search.best_estimator_._final_estimator.feature_importances_)))[::-1][0:9]]
#std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)[np.argsort(np.array(abs(model.feature_importances_)))[::-1][0:9]]



In [555]:
print(X.columns[np.argsort(np.array(abs(search.best_estimator_._final_estimator.feature_importances_)))[::-1][0:9]])

Index(['n_companies_SMOKE', 'emiss_agropec_full', 'rain1',
       'n_jobs_TRANSPORTATION', 'n_companies_TRANSPORTATION', 'emiss_agr_full',
       'n_jobs_WATER-BASED TRANSPORTATION', 'near_hidrovia', 'n_jobs_SMOKE'],
      dtype='object')


In [556]:
np.sort(np.array(abs(search.best_estimator_._final_estimator.feature_importances_)))[::-1][0:9]

array([0.34433037, 0.29893637, 0.1393934 , 0.05141435, 0.02454278,
       0.01834395, 0.01748908, 0.01225182, 0.00898887])

## Lasso Regression

5.1.1. Lasso. The LASSO regression, first introduced by (Tibshirani, 1996), is similar to a logistic regression, but adds a penalization term based on the sum of the absolute values of the coefficients. This penalization term aims at shrinking the parameters towards zero. Hence this estimator is similar to a logit model, but it is more parsimonious, adding only those variables that are relevant predictors. One of the advantages of this model is that it is simple and less prone to over-fitting. However, it is incapable of identifying complex relationships between the predictors and our outcome variable, i.e., corruption. **The tuning parameter in the cross-validation is the weight of the penalization term in the objective function (λ), which is optimized over a grid of potential values.**

For the LASSO model the importance is determined by the estimated co- efficients of the regression, where larger parameters (in absolute value) correspond to higher importance.

In [557]:
from sklearn.linear_model import Lasso

In [558]:
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',Lasso())
])

In [559]:
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.1,10,0.1)},
                      cv = 5, scoring = "neg_mean_squared_error",verbose = 3
                      )

In [560]:
search.fit(X_train,Y_train)

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV 1/5] END ................model__alpha=0.1;, score=-90.979 total time=   0.4s
[CV 2/5] END ................model__alpha=0.1;, score=-90.077 total time=   0.4s
[CV 3/5] END ................model__alpha=0.1;, score=-91.623 total time=   0.4s
[CV 4/5] END ................model__alpha=0.1;, score=-90.658 total time=   0.4s
[CV 5/5] END ................model__alpha=0.1;, score=-89.123 total time=   0.4s
[CV 1/5] END ................model__alpha=0.2;, score=-91.445 total time=   0.4s
[CV 2/5] END ................model__alpha=0.2;, score=-90.573 total time=   0.4s
[CV 3/5] END ................model__alpha=0.2;, score=-92.081 total time=   0.4s
[CV 4/5] END ................model__alpha=0.2;, score=-91.068 total time=   0.4s
[CV 5/5] END ................model__alpha=0.2;, score=-89.567 total time=   0.3s
[CV 1/5] END model__alpha=0.30000000000000004;, score=-91.755 total time=   0.3s
[CV 2/5] END model__alpha=0.30000000000000004;,

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8,
       7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1,
       9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])},
             scoring='neg_mean_squared_error', verbose=3)

In [561]:
search.best_params_

{'model__alpha': 0.1}

In [562]:
coefficients = search.best_estimator_.named_steps['model'].coef_
importance = np.abs(coefficients)


In [563]:
X.columns[np.argsort(np.array(abs( importance )))[::-1][0:9]]

Index(['illegal_industry', 'emiss_agropec_full', 'n_jobs_MACHINERY',
       'n_companies_SMOKE', 'n_companies_WOOD PROD', 'rain1', 'near_roads',
       'n_jobs_AGRICULTURE', 'n_jobs_MINING'],
      dtype='object')

In [564]:
np.sort(np.array(abs(importance)))[::-1][0:9]

array([0.52234651, 0.44754617, 0.35420192, 0.35114794, 0.28636581,
       0.27489086, 0.23303574, 0.15230478, 0.14259647])

In [565]:
#np.array(X.columns)[importance > 0]


In [566]:
#np.array(X.columns)[importance == 0]

## Gradient Boosting Machine

5.1.3. Gradient Boosting Machine. Gradient Boosting Machines (GBM) are ensembles of weak learners, in this case, decision trees. Under boosting, classification algorithms are sequentially applied to a reweighted version of the training data (Friedman et al., 2000). GBM is a variant of Random Forests, in which trees are not fitted randomly nor independently. Instead, each tree is fitted sequentially to the full dataset, in such a way that the weaknesses of trees are identified by using gradients in the loss function, allowing subsequent predictors to learn from the mistakes of the previous ones. In other words, a gradient descent procedure is used to minimize the loss when adding new trees. Consequently, as opposed to Random Forests, observations are not selected via bootstraping, but as a function of past errors. In this way, the addition of each tree offers a slight improvement in the model (Freund et al., 1999). **In our models, we keep fixed the learning rate (shrinkage parameter) and the minimum number of observations in the terminal nodes to avoid overfitting, and use cross-validation to determine the optimal number of trees and the interaction depth.**

- figure out what learning rate they used, I did 0.1
- figure out what they used for min_samples_leaf

In [567]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import GradientBoostingRegressor

In [568]:
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',GradientBoostingRegressor(learning_rate = 0.1, min_samples_leaf = 2))
])

In [569]:
search = GridSearchCV(pipeline,
                      {'model__n_estimators':np.arange(50, 150, 50), 'model__max_depth':np.arange(3, 5, 1)},
                      cv = 5, scoring = "neg_mean_squared_error",verbose = 3
                      )

In [570]:
search.fit(X_train,Y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END model__max_depth=3, model__n_estimators=50;, score=-87.412 total time=  20.5s
[CV 2/5] END model__max_depth=3, model__n_estimators=50;, score=-86.713 total time=  21.3s
[CV 3/5] END model__max_depth=3, model__n_estimators=50;, score=-88.475 total time=  20.9s
[CV 4/5] END model__max_depth=3, model__n_estimators=50;, score=-87.253 total time=  20.5s
[CV 5/5] END model__max_depth=3, model__n_estimators=50;, score=-85.908 total time=  20.4s
[CV 1/5] END model__max_depth=3, model__n_estimators=100;, score=-86.167 total time=  40.4s
[CV 2/5] END model__max_depth=3, model__n_estimators=100;, score=-85.901 total time=  40.7s
[CV 3/5] END model__max_depth=3, model__n_estimators=100;, score=-87.609 total time=  40.7s
[CV 4/5] END model__max_depth=3, model__n_estimators=100;, score=-86.183 total time=  40.1s
[CV 5/5] END model__max_depth=3, model__n_estimators=100;, score=-84.978 total time=  40.7s
[CV 1/5] END model__max_d

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model',
                                        GradientBoostingRegressor(min_samples_leaf=2))]),
             param_grid={'model__max_depth': array([3, 4]),
                         'model__n_estimators': array([ 50, 100])},
             scoring='neg_mean_squared_error', verbose=3)

In [571]:
search.best_params_

{'model__max_depth': 4, 'model__n_estimators': 100}

In [572]:
coefficients = search.best_estimator_.named_steps['model'].feature_importances_
importance = np.abs(coefficients)



In [573]:
X.columns[np.argsort(np.array(abs( importance )))[::-1][0:9]]

Index(['emiss_agropec_full', 'rain1', 'n_companies_SMOKE', 'near_roads',
       'near_mines', 'near_hidrovia', 'elevation', 'emiss_agr_full',
       'n_companies_TRANSPORTATION'],
      dtype='object')

In [574]:
np.sort(np.array(abs(importance)))[::-1][0:9]

array([0.10818899, 0.10558344, 0.10314049, 0.05996834, 0.05936298,
       0.04117405, 0.04045005, 0.03792972, 0.0282134 ])

## Neural Network

5.1.4. Neural Networks. Neural networks model the relationship between input and output signals through models that mimic the way biological brains work. In particular, neural networks are composed of three basic elements: an activation function, that for each neuron, transforms the weighted average of input signals (predictors) into an output signal; a network topology, which is composed by the number of neurons, layers, and connections used by the model; and a training algorithm, which determines the way in which connection weights are set with the task of activating or not neurons as a function of the input signals. This process determines the final prediction of the model. The most common activation functions include the logistic sigmoid, linear, saturated linear, hyperbolic tangent, and Gaussian (Radial Basis) functions. In the end, the process entails an optimization problem in which the optimal weights of the input signals are determined for each node. **In this analysis, we keep fixed a logistic activation function and use cross-validation to determine the optimal number of units in the hidden layer (size) and the regularization parameter (decay).**


In the case of neural networks, importance is determined by the weights that connect neurons within the network.

In [114]:
from sklearn.neural_network import MLPRegressor

In [115]:
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model', MLPRegressor(activation = 'logistic', random_state=42))
])

In [116]:
#search = GridSearchCV(pipeline,
#                      {'model__hidden_layer_sizes':[(50,),(100,),(150, )], 'model__alpha':np.arange(0.00001, 0.001, 0.0001)},
#                      cv = 5, scoring = "neg_mean_squared_error",verbose = 3
#                      )

In [117]:
search = GridSearchCV(pipeline,
                      {'model__hidden_layer_sizes':[(50,),(100,)], 'model__alpha':np.arange(0.00001, 0.001, 0.001)},
                      cv = 5, scoring = "neg_mean_squared_error",verbose = 3
                      )

In [118]:
search.fit(X_train,Y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] model__alpha=1e-05, model__hidden_layer_sizes=(50,) .............
[CV]  model__alpha=1e-05, model__hidden_layer_sizes=(50,), score=-191.6413519048654, total=  50.7s
[CV] model__alpha=1e-05, model__hidden_layer_sizes=(50,) .............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   50.9s remaining:    0.0s


[CV]  model__alpha=1e-05, model__hidden_layer_sizes=(50,), score=-195.51521274117843, total=  48.1s
[CV] model__alpha=1e-05, model__hidden_layer_sizes=(50,) .............


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.7min remaining:    0.0s


[CV]  model__alpha=1e-05, model__hidden_layer_sizes=(50,), score=-190.1157574529073, total=  44.0s
[CV] model__alpha=1e-05, model__hidden_layer_sizes=(50,) .............
[CV]  model__alpha=1e-05, model__hidden_layer_sizes=(50,), score=-190.8627666256797, total=  45.8s
[CV] model__alpha=1e-05, model__hidden_layer_sizes=(50,) .............
[CV]  model__alpha=1e-05, model__hidden_layer_sizes=(50,), score=-190.98240897085142, total=  45.6s
[CV] model__alpha=1e-05, model__hidden_layer_sizes=(100,) ............
[CV]  model__alpha=1e-05, model__hidden_layer_sizes=(100,), score=-181.08384632539477, total= 1.2min
[CV] model__alpha=1e-05, model__hidden_layer_sizes=(100,) ............
[CV]  model__alpha=1e-05, model__hidden_layer_sizes=(100,), score=-178.5521569859331, total= 1.2min
[CV] model__alpha=1e-05, model__hidden_layer_sizes=(100,) ............
[CV]  model__alpha=1e-05, model__hidden_layer_sizes=(100,), score=-176.8844953217453, total= 1.2min
[CV] model__alpha=1e-05, model__hidden_layer_s

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  9.7min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('model', MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learnin...       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'model__hidden_layer_sizes': [(50,), (100,)], 'model__alpha': array([1.e-05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=3)

In [119]:
search.best_estimator_.named_steps['model'].coefs_

[array([[ 0.75548786,  1.06596312, -0.04782517, ..., -0.15874169,
          2.19734343,  3.24239091],
        [-5.57644112, -0.59442137, -5.88787021, ..., -6.45570742,
         -1.72369361,  1.71062616],
        [ 1.27235407,  0.12020897,  0.46485571, ..., -0.13080582,
          0.87692592,  0.09162285],
        ...,
        [-1.4719489 , -0.24169463,  4.40726754, ...,  2.4545995 ,
          1.23165105,  3.11205949],
        [ 0.64712573, -3.86211834, -1.39325665, ...,  0.35964505,
          0.28438471, -3.70023438],
        [ 0.06914469,  2.98590669,  1.95811988, ..., -1.58893357,
         -1.57679135,  6.85888204]]), array([[  1.14687716],
        [-12.0034341 ],
        [  3.18520091],
        [  0.90896262],
        [  3.48539061],
        [  2.82077576],
        [-13.41527625],
        [  4.17703654],
        [  3.56005133],
        [  2.59546558],
        [  0.62445709],
        [  1.57678856],
        [  2.3545209 ],
        [  2.37885124],
        [  3.5759784 ],
        [  1.4

In [120]:
coefficients = search.best_estimator_.named_steps['model'].coefs_
importance = np.abs(coefficients)



In [2]:
import eli5

In [3]:

import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(search.best_estimator_.named_steps['model'], random_state=1).fit(X_test, Y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

NameError: name 'search' is not defined

In [179]:
#import shap

#explainer = shap.KernelExplainer(model.predict,X_train)

In [180]:
#from sklearn.inspection import permutation_importance

#r = permutation_importance(search.best_estimator_.named_steps['model'], X_test, Y_test,n_repeats=30, random_state=0)

In [ ]:


#for i in r.importances_mean.argsort()[::-1]:
#     if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
#         print(f"{diabetes.feature_names[i]:<8}"
#              f"{r.importances_mean[i]:.3f}"
#                             f" +/- {r.importances_std[i]:.3f}")

In [132]:
#X.columns[np.argsort(np.array(abs( importance )))[::-1][0:9]]

## Super Learner Ensemble

5.1.5. Super Learner Ensemble. Ensembles are collections of predictors which are grouped to each other in order to give a final prediction. It is usually the case that ensembles— as they result from the combination of different models—perform better than their individual components. For our analysis, we use the Super Learner ensemble method developed by Polley et al. (2011), which finds an optimal combination of individual prediction models by minimizing the cross-validated out-of-bag risk of these predictions. It has been shown that the Super Learner performs asymptotically as well as the best possible weighted combination of its constituent algorithms (Van der Laan et al., 2007). We use the Super Learner models not only to stack the individual predictions, but also to test for the relative importance of different groups of variables to predict corruption.

In [181]:
from mlens.ensemble import SuperLearner

[MLENS] backend: threading


In [182]:
#feed in the predictions for forest loss from each of the methods used above
#do I feed in the best parameters of each one? probably but double check

In [183]:
def get_models():
    models = list()
    models.append(Lasso())
    models.append(RandomForestRegressor(n_estimators=10))
    models.append(GradientBoostingRegressor(learning_rate = 0.1, min_samples_leaf = 2))
    models.append(MLPRegressor(activation = 'logistic', random_state=42))
    return models

In [ ]:
def get_out_of_fold_predictions(X, y, models):
    meta_X, meta_y = list(), list()
    
    # define split of data
    kfold = KFold(n_splits=5, shuffle=True)
    
    # enumerate splits
    for train_ix, test_ix in kfold.split(X):
        fold_yhats = list()
        # get data
        train_X, test_X = X[train_ix], X[test_ix]
        train_y, test_y = y[train_ix], y[test_ix]
        meta_y.extend(test_y)
        
        # fit and make predictions with each sub-model
        for model in models:
            model.fit(train_X, train_y)
            yhat = model.predict(test_X)
            # store columns
            fold_yhats.append(yhat.reshape(len(yhat),1))
        # store fold yhats as columns
        meta_X.append(hstack(fold_yhats))
    return vstack(meta_X), asarray(meta_y)